<a href="https://colab.research.google.com/github/YihanMou0511/Asset-prices-forecasting/blob/main/Logistic%20regression%20(credit%20fraud).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.) Import the Credit Card Fraud Data From CCLE

In [20]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import check_random_state
from scipy.sparse import hstack,vstack

In [2]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [4]:
df = pd.read_csv("/content/gdrive/MyDrive/441B/fraudTest.csv")

In [5]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# 2.) Select four columns to use as features (one just be trans_date_trans)

In [59]:
df_select=df[['trans_date_trans_time','category','amt','city_pop',"is_fraud"]]

In [60]:
df_select.columns

Index(['trans_date_trans_time', 'category', 'amt', 'city_pop', 'is_fraud'], dtype='object')

# 3.) Create a unique variable out of trans_date.

In [61]:
type(df_select['trans_date_trans_time'][0])

str

In [62]:
df_select['trans_date_trans_time']=pd.to_datetime(df_select['trans_date_trans_time'])

<ipython-input-62-9ba00a5dfdf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['trans_date_trans_time']=pd.to_datetime(df_select['trans_date_trans_time'])


In [63]:
df_select['time_var']=[i.hour for i in df_select['trans_date_trans_time']]
df_select['time_var']=[1 if i <6 else 0 for i in df_select['time_var']]

<ipython-input-63-d352c3b9b24a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['time_var']=[i.hour for i in df_select['trans_date_trans_time']]
<ipython-input-63-d352c3b9b24a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['time_var']=[1 if i <6 else 0 for i in df_select['time_var']]


In [71]:
x=pd.get_dummies(df_select,['category']).drop(['trans_date_trans_time','is_fraud'],axis=1)
y=df['is_fraud']

In [72]:
x.head()

,amt,city_pop,time_var,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,2.86,333497,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,29.84,302,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,41.28,34496,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,60.05,54767,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3.19,1126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# 4.) Oversample the data (this will be your training data). SKIP

In [82]:
#from imblearn.over_sampling import SMOTE

#sm = SMOTE(random_state=42)
#resample_x, resample_y = sm.fit_resample(x, y)

In [83]:
resample_x = x
resample_y = y

# 5.) Train a Logistic regression.

In [84]:
from sklearn.linear_model import LogisticRegression

In [85]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(resample_x)

In [86]:
log_reg = LogisticRegression().fit(X_normalized, resample_y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [87]:
from sklearn.metrics import roc_curve

# Obtain the probability of each sample belonging to the positive class
y_probs = log_reg.predict_proba(X_normalized)[:,1]

# Compute the false positive rate, true positive rate and thresholds
fpr, tpr, thresholds = roc_curve(resample_y, y_probs)

threshold = thresholds[np.argmin(np.abs(fpr - 0.05))]

print('Threshold for a false positive rate of 5% is', threshold)

Threshold for a false positive rate of 5% is 0.011009377787600649


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [88]:
df_temp = df_select.copy()

In [119]:
df_temp["pred"] = log_reg.predict(X_normalized)

In [120]:
df_temp = df_temp[["pred", "is_fraud", "amt"]]
df_temp.head()

,pred,is_fraud,amt
0,0,0,2.86
1,0,0,29.84
2,0,0,41.28
3,0,0,60.05
4,0,0,3.19


In [121]:
df_temp['plus']=df_temp['pred']+df_temp['is_fraud']
df_temp.loc[df_temp[df_temp.plus == 0].index.tolist(),'result'] = 'tn'
df_temp.loc[df_temp[df_temp.plus == 2].index.tolist(),'result'] = 'tp'
df_temp['minus']=df_temp['pred']-df_temp['is_fraud']
df_temp.loc[df_temp[df_temp.minus == 1].index.tolist(),'result'] = 'fp'
df_temp.loc[df_temp[df_temp.minus == -1].index.tolist(),'result'] = 'fn'

In [122]:
df_temp=df_temp.drop(['plus','minus'],axis=1)
df_temp

,pred,is_fraud,amt,result
0,0,0,2.86,tn
1,0,0,29.84,tn
2,0,0,41.28,tn
3,0,0,60.05,tn
4,0,0,3.19,tn
...,...,...,...,...
555714,0,0,43.77,tn
555715,0,0,111.84,tn
555716,0,0,86.88,tn
555717,0,0,7.99,tn


In [123]:
np.sum(df_temp[df_temp['result']=='tn']['amt'])*.02 - np.sum(df_temp[df_temp['result']=='fn']['amt'])

-400703.0168

The company loses 400703.

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [126]:
model = LogisticRegression(penalty='l1',solver='liblinear')

las=model.fit(X_normalized, resample_y)

In [127]:
las.coef_

array([[ 0.33373241, -0.11676236,  0.1662583 , -0.0531717 , -0.05771463,
         0.06749971,  0.        ,  0.39221745, -0.0434049 , -0.08349805,
        -0.06832004,  0.30498416, -0.00704524,  0.01439959,  0.40731787,
         0.10328285, -0.92046388]])

Most of the variables don't go to 0, so the selected features can be used in a trusted prediction model.